## Model Training

#### 1.1 Import Data and Required Packages
##### Importing Pandas, Numpy, Matplotlib, Seaborn and Warings Library.

In [29]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

#### Import the CSV Data as Pandas DataFrame

In [30]:
df = pd.read_csv('data/stud.csv')

#### Show Top 5 Records

In [31]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


#### Preparing X and Y variables

In [32]:
X = df.drop(columns=['math_score'],axis=1)

In [33]:
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [34]:
# Displaying the unique categories present in each categorical column of our dataset
categorical_columns = ['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch', 'test_preparation_course']

for column in categorical_columns:
    unique_values = df[column].unique()  # Extracting unique values for each category
    print(f"Unique categories in '{column.replace('_', ' ')}': {unique_values}")

Unique categories in 'gender': ['female' 'male']
Unique categories in 'race ethnicity': ['group B' 'group C' 'group A' 'group D' 'group E']
Unique categories in 'parental level of education': ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Unique categories in 'lunch': ['standard' 'free/reduced']
Unique categories in 'test preparation course': ['none' 'completed']


In [35]:
y = df['math_score']

In [36]:
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

In [37]:
# Construct a processing object with three distinct transformers
numeric_attributes = X.select_dtypes(exclude=["object"]).columns
categorical_attributes = X.select_dtypes(include=["object"]).columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Define transformations for numerical and categorical data
scaler_for_numerical = StandardScaler()
encoder_for_categorical = OneHotEncoder()

# Initialize ColumnTransformer with appropriate transformations
data_transforms = ColumnTransformer(
    [
        ("CategoricalEncoder", encoder_for_categorical, categorical_attributes),
        ("NumericalScaler", scaler_for_numerical, numeric_attributes),        
    ]
)


In [38]:
# Use the correct variable name for fitting and transforming
X_transformed = data_transforms.fit_transform(X)

In [39]:
X.shape

(1000, 7)

In [40]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((800, 7), (200, 7))

#### Create an Evaluate Function to give all metrics after model Training

In [41]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [42]:
# Define a dictionary to hold different regression models
regression_models = {
    "Ordinary Least Squares": LinearRegression(),
    "Lasso Regression": Lasso(),
    "Ridge Regression": Ridge(),
    "KNN Regressor": KNeighborsRegressor(),
    "Tree Regressor": DecisionTreeRegressor(),
    "Forest Regressor": RandomForestRegressor(),
    "Extreme Gradient Boosting": XGBRegressor(),
    "CatBoost Regressor": CatBoostRegressor(verbose=False),
    "Adaptive Boosting Regressor": AdaBoostRegressor()
}

# Lists to store model names and their respective R2 scores for comparison
model_names = []
test_r2_scores = []

# Iterate through the dictionary and perform actions for each model
for model_name, model_instance in regression_models.items():
    model_instance.fit(X_train, y_train)  # Fit model on training data

    # Predict using the trained model
    predictions_on_train = model_instance.predict(X_train)
    predictions_on_test = model_instance.predict(X_test)
    
    # Evaluate the model's performance on both training and test sets
    train_mae, train_rmse, train_r2 = evaluate_model(y_train, predictions_on_train)
    test_mae, test_rmse, test_r2 = evaluate_model(y_test, predictions_on_test)

    # Log the model name
    print(model_name)
    model_names.append(model_name)
    
    # Display performance metrics for the training set
    print('Training Set Performance')
    print("- RMSE: {:.4f}".format(train_rmse))
    print("- MAE: {:.4f}".format(train_mae))
    print("- R^2: {:.4f}".format(train_r2))

    print('----------------------------------')
    
    # Display performance metrics for the test set
    print('Test Set Performance')
    print("- RMSE: {:.4f}".format(test_rmse))
    print("- MAE: {:.4f}".format(test_mae))
    print("- R^2: {:.4f}".format(test_r2))
    test_r2_scores.append(test_r2)
    
    print('='*35)
    print('\n') 


ValueError: could not convert string to float: 'female'

### Results

In [ ]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

## Linear Regression

In [ ]:
lin_model = LinearRegression(fit_intercept=True)
lin_model = lin_model.fit(X_train, y_train)
y_pred = lin_model.predict(X_test)
score = r2_score(y_test, y_pred)*100
print(" Accuracy of the model is %.2f" %score)

## Plot y_pred and y_test

In [ ]:
plt.scatter(y_test,y_pred);
plt.xlabel('Actual');
plt.ylabel('Predicted');

In [ ]:
sns.regplot(x=y_test,y=y_pred,ci=None,color ='red');

#### Difference between Actual and Predicted Values

In [ ]:
pred_df=pd.DataFrame({'Actual Value':y_test,'Predicted Value':y_pred,'Difference':y_test-y_pred})
pred_df